In [1]:
# Set up the client object

from garminconnect import Garmin
from retrieve_creds import retrieve_creds

username, password = retrieve_creds('garmin connect/explore')
client = Garmin(username, password)
client.login()

print("Done")

Done


In [2]:
filename = 'Lakewood_Running_17836250937.gpx'
gpx = client.download_activity("17836250937", Garmin.ActivityDownloadFormat.GPX)

# with open(filename, 'wb') as f:
#     f.write(gpx)
#     print("Downloaded activity to", filename)

# Convert the bytes to a string
gpx = gpx.decode("utf-8")


print("Done")

Done


In [4]:
import gpxpy
import gpxpy.gpx
from geopy.distance import geodesic
gpx = gpxpy.parse(gpx)

In [5]:


# # Load the GPX file
# with open('Lakewood_Running_17836250937.gpx', 'r') as gpx_file:
#     gpx = gpxpy.parse(gpx_file)

# Initialize total distance
total_distance = 0.0

# Iterate through track points and calculate distance
for track in gpx.tracks:
    for segment in track.segments:
        for i in range(1, len(segment.points)):
            point1 = segment.points[i - 1]
            point2 = segment.points[i]
            coords_1 = (point1.latitude, point1.longitude)
            coords_2 = (point2.latitude, point2.longitude)
            total_distance += geodesic(coords_1, coords_2).miles

print(f"Total distance: {total_distance:.2f} miles")

Total distance: 6.24 miles


In [6]:

from datetime import datetime



# Initialize total distance and start time
total_distance = 0.0
start_time = None
elapsed_time_5k = None

# Iterate through track points and calculate distance
for track in gpx.tracks:
    for segment in track.segments:
        for i in range(1, len(segment.points)):
            point1 = segment.points[i - 1]
            point2 = segment.points[i]
            coords_1 = (point1.latitude, point1.longitude)
            coords_2 = (point2.latitude, point2.longitude)
            distance = geodesic(coords_1, coords_2).kilometers
            total_distance += distance

            # Record the start time
            if start_time is None:
                start_time = point1.time

            # Check if the total distance has reached 5 kilometers
            if total_distance >= 5.0 and elapsed_time_5k is None:
                elapsed_time_5k = point2.time - start_time
                break

        if elapsed_time_5k is not None:
            break
    if elapsed_time_5k is not None:
        break

if elapsed_time_5k is not None:
    print(f"Elapsed time to reach 5 kilometers: {elapsed_time_5k}")
else:
    print("5 kilometers not reached in the provided GPX data.")

Elapsed time to reach 5 kilometers: 0:29:39


In [ ]:
# Read in from Garmin 945 Activities Raw.pkl in the current directory

# alright... you're going to need the GPX fo all runs you've done. And then pull out the elapsed time from that... 

import pickle
import pandas as pd

# Read in the data
with open('Garmin 945 Activities Raw.pkl', 'rb') as f:
    activity_data = pickle.load(f)


# filter activityType down to these activities: Running, Treadmill Running, Trail Running
activity_data['activityType'] = activity_data['activityType'].apply(lambda x: x.get('typeKey') if isinstance(x, dict) else None)
activity_data = activity_data[activity_data['activityType'].isin(['running', 'treadmill_running', 'Trail_running'])]

# subset cols to just the activityId and activityName (convert both to strings, first)
activity_data['activityId'] = activity_data['activityId'].astype(str)
activity_data['activityName'] = activity_data['activityName'].astype(str)
activity_data = activity_data[['activityId', 'activityName', 'activityType']]

# Create a gpxFileName column that is activityName_activity_id.gpx
activity_data['gpxFileName'] = activity_data['activityName'] + '_' + activity_data['activityId'] + '.gpx'
activity_data['gpxFileName'] = activity_data['gpxFileName'].str.replace(' ', '_')

x = activity_data.gpxFileName[0:1]
print(x)






In [ ]:
import os
from garminconnect import Garmin, GarminConnectConnectionError, GarminConnectTooManyRequestsError

# Replace with your Garmin Connect credentials
USERNAME = "your_email"
PASSWORD = "your_password"

def download_gpx(activity_id, filename):
    """
    Downloads the GPX file for a specific activity.
    """
    try:
        with open(filename, "wb") as gpx_file:
            gpx_file.write(api.download_activity(activity_id, "gpx"))
        print(f"GPX file saved as {filename}")
    except Exception as e:
        print(f"Error downloading GPX for activity {activity_id}: {e}")

try:
    # Login to Garmin Connect
    api = Garmin(USERNAME, PASSWORD)
    api.login()

    # Fetch recent activities
    print("Fetching activities...")
    activities = api.get_activities(0, 10)  # Fetch 10 activities; adjust as needed

    # Process each activity
    for activity in activities:
        activity_id = activity["activityId"]
        activity_name = activity.get("activityName", "Activity")
        gpx_filename = f"{activity_name}_{activity_id}.gpx"
        gpx_filename = gpx_filename.replace(" ", "_")  # Replace spaces with underscores for filename safety
        print(f"Downloading GPX for activity {activity_name} (ID: {activity_id})...")
        download_gpx(activity_id, gpx_filename)

except GarminConnectConnectionError as conn_err:
    print(f"Connection error: {conn_err}")
except GarminConnectTooManyRequestsError as rate_err:
    print(f"Too many requests: {rate_err}")
except Exception as e:
    print(f"An error occurred: {e}")
